# Deep Learning for Computer Vision
***
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/498_FA2019_lecture01.pdf_page_10.png?raw=true)
## Plan for this lecture
1. **Deep Learning** ~~for Computer Vision~~
2. **Deep Learning for Computer Vision**
***

## Preparation

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
print('ready.')

***

## 1. **Deep Learning** ~~for Computer Vision~~
### 1.1  Recap:Deep Learning
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/498_FA2019_lecture01.pdf_page_09.jpg?raw=true)
***

#### 1.1.1  Real neuron
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide2.PNG?raw=true)
***

#### 1.1.1  Real neuron
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide90.PNG?raw=true)
***

#### 1.1.2  Artificial neuron
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide3.PNG?raw=true)
***

#### 1.1.2  Artificial neuron
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide4.PNG?raw=true)
***
    

In [ ]:
# for reproducibility
torch.manual_seed(777)

# input
X = torch.FloatTensor([[0, 0], [1, 1]])
Y = torch.FloatTensor([[0],[1]])

# nn layers
input_layer = torch.nn.Linear(2, 1, bias=True)
sigmoid = torch.nn.Sigmoid()

# model
model = torch.nn.Sequential(input_layer, sigmoid)

# define cost/loss & optimizer
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1)  # modified learning rate from 0.1 to 1

for step in range(10001):
    optimizer.zero_grad()
    hypothesis = model(X)

    # cost/loss function
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 1000 == 0:
        print(step, cost.item())

# for drawing
model_params = list(model.parameters())
model_weights = model_params[0].data.numpy()
model_bias = model_params[1].data.numpy()

plt.scatter(X.numpy()[[0,-1], 0], X.numpy()[[0, -1], 1], s=50)
plt.scatter(X.numpy()[[1,-1], 0], X.numpy()[[1, -1], 1], c='red', s=50)

x_1 = np.arange(-0.1, 1.1, 0.1)
y_1 = ((x_1 * model_weights[0,0]) + model_bias[0]) / (-model_weights[0,1])
plt.plot(x_1, y_1)

plt.legend(["neuron_1"], loc=8)
plt.show()

#### 1.1.3 Limitations
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide6.PNG?raw=true)
***

In [ ]:
torch.manual_seed(777)

X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = torch.FloatTensor([[0], [1], [1], [0]])

input_layer = torch.nn.Linear(2, 1, bias=True)
sigmoid = torch.nn.Sigmoid()
model = torch.nn.Sequential(input_layer, sigmoid)

criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1)  # modified learning rate from 0.1 to 1

for step in range(10001):
    optimizer.zero_grad()
    hypothesis = model(X)

    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 1000 == 0:
        print(step, cost.item())

model_params = list(model.parameters())
model_weights = model_params[0].data.numpy()
model_bias = model_params[1].data.numpy()

plt.scatter(X.numpy()[[0,-1], 0], X.numpy()[[0, -1], 1], s=50)
plt.scatter(X.numpy()[[1,2], 0], X.numpy()[[1, 2], 1], c='red', s=50)

x_1 = np.arange(-0.1, 1.1, 0.1)
y_1 = ((x_1 * model_weights[0,0]) + model_bias[0]) / (-model_weights[0,1])
plt.plot(x_1, y_1)

plt.legend(["neuron_1"], loc=8)
plt.show()

#### 1.1.4  First AI Winter
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/ai_winter.png?raw=true)
***

### 1.2 Multi-layer Perceptron
#### 1.2.1 XOR Problem
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/xor.png?raw=true)
***

#### 1.2.2 Solving XOR Problem
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide11.PNG?raw=true)
***

In [ ]:
# for reproducibility
torch.manual_seed(777)

X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = torch.FloatTensor([[0], [1], [1], [0]])

# nn layers
input_layer = torch.nn.Linear(2, 2, bias=True)
hidden = torch.nn.Linear(2, 1, bias=True)
sigmoid = torch.nn.Sigmoid()

# model
model = torch.nn.Sequential(input_layer, sigmoid, hidden, sigmoid)

# define cost/loss & optimizer
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1)  # modified learning rate from 0.1 to 1

for step in range(10001):
    optimizer.zero_grad()
    hypothesis = model(X)

    # cost/loss function
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 1000 == 0:
        print(step, cost.item())

model_params = list(model.parameters())
model_weights = model_params[0].data.numpy()
model_bias = model_params[1].data.numpy()

plt.scatter(X.numpy()[[0,-1], 0], X.numpy()[[0, -1], 1], s=50)
plt.scatter(X.numpy()[[1,2], 0], X.numpy()[[1, 2], 1], c='red', s=50)

x_1 = np.arange(-0.1, 1.1, 0.1)
y_1 = ((x_1 * model_weights[0,0]) + model_bias[0]) / (-model_weights[0,1])
plt.plot(x_1, y_1)

x_2 = np.arange(-0.1, 1.1, 0.1)
y_2 = ((x_2 * model_weights[1,0]) + model_bias[1]) / (-model_weights[1,1])
plt.plot(x_2, y_2)

plt.legend(["neuron_1", "neuron_2"], loc=8)
plt.show()

#### 1.2.3  Multi-layer Perceptron
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide13.PNG?raw=true)
***

#### 1.2.3  Multi-layer Perceptron
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide14.PNG?raw=true)
***

#### 1.2.3  Multi-layer Perceptron
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide15.PNG?raw=true)
***

#### 1.2.3  Multi-layer Perceptron
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide16.PNG?raw=true)
***

#### 1.2.3  Multi-layer Perceptron
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide17.PNG?raw=true)
[Multi-layer Perceptron](https://playground.tensorflow.org)
***

### 1.3  Stochastic Gradient Decent
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/sgd.png?raw=true)
***

### 1.4 Vanishing Gradient
#### 1.4.1  Second AI Winter
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/ai_winter.png?raw=true)
***

### 1.4.2 Vanishing Gradient Problem
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide22.PNG?raw=true)
***

#### 1.4.2 Dropout
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/drop_out.png?raw=true)
***

### 1.5 What is the DL?
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide25.PNG?raw=true)
***
